In [35]:
###
### Import libraries 
###
import PyPDF2
import sklearn
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sb
from sklearn import preprocessing
import numpy as np

In [33]:
# Open the PDF file
with open('./papers/Cernelev-TongueNav-Manuscript-ImNO2023-v4.pdf', 'rb') as file:
    pdf = PyPDF2.PdfReader(file)
    
    # Initialize an empty string to store the text
    text = ''
    
    # Iterate through all the pages
    for i in range(len(pdf.pages)):
        page = pdf.pages[i]
        text += page.extract_text()
        
    # Write the text to a file
    with open('research_paper.txt', 'w', encoding='utf-8') as text_file:
        text_file.write(text)

In [31]:
text = text[120:]
print(text)

 The surgical excision of tongue tumors , referred to as g lossectomy , is a common 
treatment for oral tongue cancers. The procedure  involves excising the tumor while preserving as much of 
the surrounding healthy tissue as possible. The excised tongue tumor is sent to a pathologist to analyze the 
tissue , negative margins indicat e that the tumor was sufficiently removed from the patient  [1]. The 
computer -navigation system,  NaviKnife , uses real -time electromagnetic tracking to determine the location 
of the surgical tools  relative to the patient anatomy , originally developed for  breast -conserving surgery.  
When tested the system resulted in  the resection of less healthy tissue as well as increased confidence for 
the surgeons  [2]. In this work, we present the technical challenges and potential solutions when  translating  
previously designed  computer navigated system s to glossectomy . 
METHODS:  The NaviKnife system was  initially tested for 
glossectomy by  directl

In [34]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 &(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWY[]abcdefghijklmnopqrstuvwxyz‐–’
76


In [17]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))




[54, 55, 55, 1, 66, 54, 51, 64, 51]


In [18]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([4282]) torch.int64
tensor([21, 50, 47, 62, 66, 55, 60, 53,  1,  1, 51, 58, 51, 49, 66, 64, 61, 59,
        47, 53, 60, 51, 66, 55, 49,  1, 66, 61, 61, 58,  1, 66, 64, 47, 49, 57,
        55, 60, 53,  1, 52, 61, 64,  1, 67, 58, 66, 64, 47, 65, 61, 67, 60, 50,
         1,  6, 53, 67, 55, 50, 51, 50,  1,  1,  0,  1, 61, 64, 47, 58,  1, 49,
        47, 68, 55, 66, 71,  1, 49, 47, 60, 49, 51, 64,  1, 64, 51, 65, 51, 49,
        66, 55, 61, 60,  1,  1,  0, 36, 47, 68, 51, 58,  1,  6, 24, 67, 59, 55,
        66, 64, 67,  1, 23, 51, 64, 60, 51, 58, 51, 68, 10,  5,  1, 31, 64, 55,
        65, 66, 61, 52,  1, 33, 61, 53, 47, 11,  5, 12,  5,  1, 23, 54, 64, 55,
        65,  1, 44, 51, 67, 60, 53, 10,  5,  1, 40, 47, 59, 47, 65,  1, 28, 47,
        55, 50, 51, 53, 53, 51, 64, 11,  5,  1, 27, 51, 64, 60, 61, 66,  1, 31,
        64, 61, 60, 64, 51, 55, 52, 13,  5,  1,  1,  0, 40, 47, 59, 47, 65,  1,
        41, 60, 53, 55, 10,  5,  1, 27, 47, 48, 61, 64,  1, 26, 55, 49, 54, 66,
        5

In [19]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


In [20]:
block_size = 8
train_data[:block_size+1]

tensor([21, 50, 47, 62, 66, 55, 60, 53,  1])

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]


In [24]:
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

